In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import re

# Подготовим промежуточные файлы. Если они есть, то этот ход пропускаем!

In [ ]:
def prepare_bs(f_name): #без csv

    f = f_name + '.csv'
    df = pd.read_csv(f, sep='\t', encoding='utf8')

    df.description = df.description.map(lambda x: BeautifulSoup(x, 'lxml').get_text())
    df.description = df.description.map(lambda x: x.lower())
    df.name = df.name.map(lambda x: x.lower())

    f_name_bs = f_name + '_bs.csv'
    df.to_csv(f_name_bs, sep='\t', index=False)
    
    return df

In [ ]:
train=prepare_bs('train')
test=prepare_bs('test')

# Формируем чистые лемматизированные наборы слов

In [6]:
# склеиваем name name description (name взяли дважды, чтобы подчеркнуть, что это важные слова заголовка)
train['text']=train['name']+'. '+train['name']+'. '+train['description']
test['text']=test['name']+'. '+test['name']+'. '+test['description']

In [7]:
# Приводим к нормальной форме
import pymorphy2
m2 = pymorphy2.MorphAnalyzer()

In [ ]:
train['text_norm']=''

In [ ]:
for ii, text in enumerate(train['text']) :
    if train.loc[ii,'text_norm'] in ['', np.nan, 'nan']:
        new_text =''
        words = re.findall("([А-ЯЁа-яё]+)", text)
        for w in words:
            if w not in words_lem.keys():
                wl=m2.normal_forms(w)[0]
                if wl=='':
                    wl=w
                words_lem.update({w:wl})
            wl = words_lem[w]
            new_text = new_text + ' ' + wl
        train.loc[ii,'text_norm']=new_text.strip()
    if ii%10000 == 0 :print(ii, len(words_lem))

In [23]:
test['text_norm']=''

In [ ]:
for ii, text in enumerate(test['text']) :
    if test.loc[ii,'text_norm']  in ['', np.nan, 'nan']:
        new_text =''
        words = re.findall("([А-ЯЁа-яё]+)", text)
        for w in words:
            if w not in words_lem.keys():
                wl=m2.normal_forms(w)[0]
                if wl=='':
                    wl=w
                words_lem.update({w:wl})
            else : wl = words_lem[w]
            new_text = new_text + ' ' + wl
        test.loc[ii,'text_norm']=new_text.strip()
    if ii%10000 == 0 :print(ii, len(words_lem))

# Записываем долгожданные результаты в файлы

# Убираем лишнее, добавляем ненужное ))

In [6]:
train=train.drop(['name','description', 'text'], axis=1)
test=test.drop(['name','description', 'text', 'id'], axis=1)

In [8]:
train.text_norm=train.text_norm.fillna('')
test.text_norm=test.text_norm.fillna('')

# Собираем новый столбик, куда помещаем наборы (листы) слов

In [ ]:
train['words']=train['text_norm'].map(lambda x: list(x.split()))
test['words']=test['text_norm'].map(lambda x: list(x.split()))

# Обучаем модель Word2Vec

In [11]:
sentences = list(train['words']) + list(test['words'])

In [15]:
from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 2min 7s


In [16]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

# Считаем вектора для кажого текста

In [17]:
def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [18]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [23]:
train_like_word_list=list(train['words'])

In [24]:
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [25]:
test_like_word_list=list(test['words'])

In [26]:
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


# Всё готово! Учим RFC и считаем ответы

In [27]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train["target"])

# Формируем выходной файл

In [28]:
subm = forest.predict_proba(test_vecs)[:,1]
np.shape(subm)

(170179,)

In [29]:
fl_subm = pd.read_csv('sampleSubmission.csv', sep=',', encoding='utf8')
fl_subm['target'] = subm
fl_subm.head()

,id,target
0,200000,0.84
1,200001,1.00
2,200002,0.99
3,200003,1.00
4,200004,0.08


In [30]:
new_submit_file='my_subm_4.csv'
fl_subm.to_csv(new_submit_file, index=False)

In [31]:
subm_final=pd.read_csv(new_submit_file)
subm_final.head()

,id,target
0,200000,0.84
1,200001,1.00
2,200002,0.99
3,200003,1.00
4,200004,0.08
